In [7]:
import krpc
connection = krpc.connect(
    name="Ascent Guidance",
)

In [8]:
from krpc.services.spacecenter import Vessel
from krpc.services import spacecenter
from krpc.stream import Stream

from threading import Thread
from time import sleep

TARGET_APOAPSIS:int = float(input("Enter Target Apoapsis: "))
TARGET_INCLINE: float = float(input("Enter Target Inclination: "))

# Preparations

In [9]:
activeVessel: Vessel = connection.space_center.active_vessel

altitudeStream: Stream = connection.add_stream(getattr, activeVessel.flight(), 'mean_altitude')
apoapsisStream: Stream = connection.add_stream(getattr, activeVessel.orbit, 'apoapsis_altitude')
periapsisStream: Stream = connection.add_stream(getattr, activeVessel.orbit, 'periapsis_altitude')

# Launch Pad

In [10]:
def Activate_First_Stage_Engines() -> None:
    # Find non SRB engines
    nonSRBEngines: list[spacecenter.Engine] = [
        nonSRB for nonSRB in [

            # Find first stage engines
            part.engine for part in activeVessel.parts.in_stage(

                # Determine first stage number, indexing from 0
                stage=activeVessel.control.current_stage - 1
            ) if part.engine is not None
        ] if nonSRB.can_shutdown
    ]

    # Set throttle to 0
    activeVessel.control.throttle = 0

    # Activate engines
    for engine in nonSRBEngines:
        engine.active = True

    # Throttle up
    while activeVessel.control.throttle < 1:
        activeVessel.control.throttle += 0.05
        sleep(0.1)

# Countdown
for time in range(5, 0, -1):
    if time == 3: 
        throttleThread = Thread(target=Activate_First_Stage_Engines)
        throttleThread.start()
        print("T - 3, Activating engines...")

    else: print(f"T - {time}")

    sleep(1)

# Release clamps
activeVessel.control.activate_next_stage()
print("T - 0, Liftoff!")

T - 5
T - 4
T - 3, Activating engines...
T - 2
T - 1
T - 0, Liftoff!


# Ascent

In [11]:
def Gravity_Turn_Control() -> None:
    # Calculate target pitch angle
    targetPitchAngle: float = 90 - (
        abs(apoapsisStream() / TARGET_APOAPSIS) * 90
    )

    # Adjust pitch
    activeVessel.auto_pilot.target_pitch = targetPitchAngle

def Staging_Control() -> None:
    # Get current the engines of the current stage
    engines: list[spacecenter.Engine] = [
        part.engine for part in activeVessel.parts.in_stage(
            stage=activeVessel.control.current_stage
        ) if part.engine is not None
    ]
    fairings: list[spacecenter.Fairing] = [
        part.fairing for part in activeVessel.parts.in_stage(
            stage=activeVessel.control.current_stage
        ) if part.fairing is not None
    ]

    # if there are no engines, stage
    if len(engines) == 0 and len(fairings) == 0:
        activeVessel.control.activate_next_stage()

    # if engine is producing no thrust, stage
    for engine in engines:
        if engine.thrust <= 0:
            activeVessel.control.activate_next_stage()
            break

# Lock Control
activeVessel.auto_pilot.engage()
activeVessel.control.sas = True
activeVessel.control.rcs = True

# Clear Tower
print("Clearing Tower...")
while altitudeStream() < 300:
    activeVessel.auto_pilot.target_pitch = 90
    sleep(0.1)

# Heading Adjustment
print("Adjusting Heading...")
activeVessel.auto_pilot.target_heading = TARGET_INCLINE

# Main Guidance Loop
print("Conducting Gravity Turn...")
while apoapsisStream() < TARGET_APOAPSIS:
    Gravity_Turn_Control()
    Staging_Control()

# Engine Cut Off
activeVessel.control.throttle = 0

# Unlock Control
activeVessel.auto_pilot.disengage()
activeVessel.control.sas = False
activeVessel.control.rcs = False

Clearing Tower...
Adjusting Heading...
Conducting Gravity Turn...


# Circularization

In [12]:
def Get_Apoapsis_UT() -> float:
    """
    Returns the **Universal Time** at which the vessel will reach Apoapsis

    Returns:
        float: **Universal Time**, in seconds, at which the vessel will reach Apoapsis
    """
    currentUT: float = connection.space_center.ut
    timeTilApoapsis: float = activeVessel.orbit.time_to_apoapsis
    return currentUT + timeTilApoapsis

def In_Range(value: float, center: float, range: float) -> bool:
    """
    Determines if a value is within a range of a central value.\n
    As in, `value` within +/- `range` of `center`

    Parameters:
        value (float): The value to be checked
        center (float): The central value
        range (float): The range of the central value

    Returns:
        bool: True if the value is within the range of the central value
    """
    return True if (center - range < value) and (value < center + range) else False

def Calc_Basic_Accel() -> float:
    """
    Returns the **Basic Acceleration** of the vessel

    Returns:
        float: Acceleration, in m/s**2
    """
    maxThurst: float = activeVessel.available_thrust
    mass: float = activeVessel.mass
    return maxThurst / mass

def Create_Hills() -> list[dict[str, float]]:
    margins: list[int] = [-1, 1]
    hills: list[dict[str, float]] = []
    for time in margins:
        hills.append({
            "time": time,
            "prograde": 0,
            "normal": 0,
            "radial": 0
        })
    for prograde in margins:
        hills.append({
            "time": 0,
            "prograde": prograde,
            "normal": 0,
            "radial": 0
        })
    for normal in margins:
        hills.append({
            "time": 0,
            "prograde": 0,
            "normal": normal,
            "radial": 0
        })
    for radial in margins:
        hills.append({
            "time": 0,
            "prograde": 0,
            "normal": 0,
            "radial": radial
        })
    return hills

def Calc_Eccentricity(baseNode: dict[str, float], changes: dict[str, float]) -> float:
    node: spacecenter.Node = activeVessel.control.add_node(
        ut=baseNode["time"] + changes["time"],
        prograde=baseNode["prograde"] + changes["prograde"],
        normal=baseNode["normal"] + changes["normal"],
        radial=baseNode["radial"] + changes["radial"]
    )
    ecc: float = node.orbit.eccentricity
    node.remove()
    return ecc

def Improve_Node(baseNode: spacecenter.Node) -> spacecenter.Node:
    # Save details and delete node
    baseNodeDetails: dict[str, float] = {
        "time": baseNode.ut,
        "prograde": baseNode.prograde,
        "normal": baseNode.normal,
        "radial": baseNode.radial
    }
    baseNode.remove()

    # Create hill list
    hills: list[dict[str, float]] = Create_Hills()

    # Find best node
    bestEcc: float = 100
    bestChanges: dict[str, float] = None
    for changes in hills:
        ecc: float = Calc_Eccentricity(baseNodeDetails, changes)
        if ecc < bestEcc:
            bestEcc = ecc
            bestChanges = changes

    return activeVessel.control.add_node(
        ut=baseNodeDetails["time"] + bestChanges["time"],
        prograde=baseNodeDetails["prograde"] + bestChanges["prograde"],
        normal=baseNodeDetails["normal"] + bestChanges["normal"],
        radial=baseNodeDetails["radial"] + bestChanges["radial"]
    )
    
 
# Initialize maneuver node
baseNode: spacecenter.Node = activeVessel.control.add_node(
    ut=connection.space_center.ut+10,
)

# Hill climb to target orbit
while not In_Range(baseNode.orbit.periapsis_altitude, TARGET_APOAPSIS, 1000):
    # Improve node
    baseNode = Improve_Node(baseNode)
    if baseNode.orbit.eccentricity < 0.005:
        print(f"Target Eccentricity: {baseNode.orbit.eccentricity}")
        break

# Lock Control
activeVessel.auto_pilot.engage()
activeVessel.control.sas = True
activeVessel.control.rcs = True

# Coast to T-(0.5 * burnTime)
totalBurnTime: float = baseNode.delta_v / Calc_Basic_Accel()
print(f"Estimated burn time: {totalBurnTime} seconds")
burnStartUT: float = baseNode.ut - 0.5 * totalBurnTime
connection.space_center.warp_to(burnStartUT)

# Point to node
activeVessel.auto_pilot.target_direction = baseNode.direction(activeVessel.surface_reference_frame)
activeVessel.auto_pilot.wait()

# Do Burn and Update vector
activeVessel.control.throttle = 1
sleep(0.1)
while not In_Range(periapsisStream(), TARGET_APOAPSIS, 1000) and In_Range(apoapsisStream(), TARGET_APOAPSIS, 1000):
    # Point to node
    activeVessel.auto_pilot.target_direction = baseNode.direction(
        activeVessel.surface_reference_frame
    )

    # Check staging
    Staging_Control()
baseNode.remove()

# Orbit completion
activeVessel.control.throttle = 0
activeVessel.auto_pilot.disengage()
activeVessel.control.sas = False
activeVessel.control.rcs = False

print("Orbit Complete!")

Target Eccentricity: 0.004633301321653334
Estimated burn time: 33.455881514423076 seconds
Orbit Complete!
